In [9]:

# -*- coding: utf-8 -*-
"""
Geografische Visualisierung der KMeans-Cluster mit Rücktransformation und Zentrenanzeige
"""

# -------------------------------------------
#  Bibliotheken importieren
# -------------------------------------------
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import geopandas as gpd                        # Für die Arbeit mit Geodaten (Punkte, Polygone, Karten)
import matplotlib.pyplot as plt                # Für Visualisierung mit Karten und Geometrien
import matplotlib.patches as mpatches          # Für manuelle Legenden in Plots
from shapely.geometry import Point             # Für die Erstellung von Punktgeometrien
import joblib                                  # Zum Laden des Modells und Scalers
import pandas as pd                            # Für den Umgang mit tabellarischen Daten

# -------------------------------------------
# 1. Modelle und Daten laden
# -------------------------------------------

# Lade das trainierte KMeans-Modell
kmeans = joblib.load('kmeans_model.pkl')

# Lade kodierte Daten (zuvor transformierte Eingabedaten)
df_encoded = pd.read_csv("k_means_ohneBEZ_LOR_encoded_data.csv")

# Lade StandardScaler zur Rücktransformation der Koordinaten
scaler_coords = joblib.load('k_means_std_scaler.pkl')
scaler_coords.mean_ = scaler_coords.mean_[4:6]     # Extrahiere nur die Koordinaten-Mittelwerte
scaler_coords.scale_ = scaler_coords.scale_[4:6]   # Extrahiere nur die Skalierungsfaktoren

# Rücktransformation der Koordinaten auf Originalwerte (für Kartenvisualisierung)
df_encoded[['XGCSWGS84', 'YGCSWGS84']] = scaler_coords.inverse_transform(
    df_encoded[['XGCSWGS84', 'YGCSWGS84']]
)

# -------------------------------------------
# 2. Geodaten vorbereiten
# -------------------------------------------

# Erstelle Punkt-Geometrien aus den Koordinaten
geometry = [Point(xy) for xy in zip(df_encoded['XGCSWGS84'], df_encoded['YGCSWGS84'])]

# Erstelle ein GeoDataFrame mit WGS84-Koordinatenbezug
gdf = gpd.GeoDataFrame(df_encoded, geometry=geometry, crs="EPSG:4326")

# Cluster-Labels aus dem Modell hinzufügen
df_encoded["Cluster_Original"] = kmeans.labels_
gdf['cluster'] = df_encoded["Cluster_Original"]

# -------------------------------------------
# 3. Cluster-Zentren zurücktransformieren
# -------------------------------------------

# Indizes der Koordinatenspalten im Datenarray ermitteln
coord_columns = ['XGCSWGS84', 'YGCSWGS84']
coord_indices = [df_encoded.columns.get_loc(col) for col in coord_columns]

# Nur die Koordinaten der Clusterzentren extrahieren und zurücktransformieren
cluster_centers_scaled = kmeans.cluster_centers_[:, coord_indices]
cluster_centers_unscaled = scaler_coords.inverse_transform(cluster_centers_scaled)

# In GeoDataFrame umwandeln
df_cluster_centers = pd.DataFrame(cluster_centers_unscaled, columns=coord_columns)
cluster_centers_geometry = [Point(xy) for xy in zip(df_cluster_centers['XGCSWGS84'], df_cluster_centers['YGCSWGS84'])]
gdf_cluster_centers = gpd.GeoDataFrame(df_cluster_centers, geometry=cluster_centers_geometry, crs="EPSG:4326")

# -------------------------------------------
# 4. Bezirksgrenzen laden
# -------------------------------------------

# Lade GeoJSON-Datei mit den Berliner Bezirksgrenzen
bez_gdf = gpd.read_file("bezirksgrenzen.geojson", engine='pyogrio')

# -------------------------------------------
# 5. Visualisierung pro Cluster
# -------------------------------------------

# Alle vorhandenen Cluster (numerisch sortiert)
unique_clusters = sorted(gdf['cluster'].unique())

for cluster in unique_clusters:
    # Filter für Datenpunkte im aktuellen Cluster
    cluster_data = gdf[gdf['cluster'] == cluster]

    # Nur das Zentrum dieses Clusters extrahieren
    cluster_center = gdf_cluster_centers.iloc[[cluster]]

    # Neues Plot-Fenster
    fig, ax = plt.subplots(figsize=(12, 10))

    # Bezirksgrenzen als Hintergrundkarte zeichnen
    bez_gdf.plot(ax=ax, color='white', edgecolor='black', linewidth=0.8, alpha=0.7)

    # Dynamische Farbgebung für Clusterpunkte (z. B. viridis-Farbskala)
    cluster_color = plt.cm.viridis(cluster / len(unique_clusters))

    # Zeichne alle Punkte dieses Clusters
    cluster_data.plot(ax=ax, color=cluster_color, markersize=5, alpha=0.6, label=f"Cluster {cluster}")

    # Zeichne das Zentrum des Clusters (rot, als X)
    cluster_center.plot(ax=ax, color='red', markersize=100, marker='x', label='Cluster-Zentrum')

    # Manuelle Legende mit Patches
    legend_patches = [
        mpatches.Patch(color=cluster_color, label=f"Cluster {cluster}"),
        mpatches.Patch(color="red", label="Cluster-Zentrum")
    ]
    ax.legend(handles=legend_patches, title="Cluster", loc="upper right")

    # Titel setzen
    ax.set_title(f"K-Means Cluster {cluster}", fontsize=16)

    # Achsenbegrenzung auf Gesamtkarte setzen
    ax.set_xlim([bez_gdf.total_bounds[0], bez_gdf.total_bounds[2]])
    ax.set_ylim([bez_gdf.total_bounds[1], bez_gdf.total_bounds[3]])

    # Speichern des Plots für das Cluster
    plt.savefig(f"output/plots/cluster_{cluster}_visualisierung_mit_zentrum.png", dpi=100, bbox_inches="tight")
    plt.close()